## Препроцессинг

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [ ]:
labels = []
images = []
for cultures in glob("dataset/*"):
    for disease in glob(cultures+"/*"):
        for s in glob(disease+"/*"):
            img = img_to_array(load_img(s, target_size=(224,224))) 
            labels.append(disease.split('\\')[-1])
            images.append(img)


In [ ]:
labels[1]

In [ ]:
array_to_img(images[1])

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
import tensorflow as tf

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3)) # загружаем предобученную сеть

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
n_labels = le.fit_transform(labels)
y = to_categorical(n_labels)
y

In [ ]:
labels_count = np.unique(n_labels).size
labels_count

## Сборка модели

In [ ]:
output = restnet.layers[-1].output # используем выход пердыдущего слоя
output = tf.keras.layers.Flatten()(output) # видимо обертка
restnet = Model(restnet.input, outputs=[output]) # создаем это все дело как модель
for layer in restnet.layers: # замораживаем предобученные слои модели
    layer.trainable = False

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
model = Sequential() #  собираем сеть из предобученныз слов и полносвязных слоев с отсечением в 0.3
model.add(restnet)
model.add(Dense(512, activation='relu', input_dim=(100352)))
model.add(Dropout(0.3))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(labels_count, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', # лейблы кодируются в one-hot
              optimizer=optimizers.RMSprop(lr=2e-5), # sgd с тюнингом
              metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
[X_train, X_test, y_train, y_test] = train_test_split(X, y, test_size=0.1, train_size=0.9, random_state=777)

## Обучение

In [ ]:
history = model.fit(X_train, y_train,
                              steps_per_epoch=100, 
                              epochs=2,
                              validation_data=(X_test, y_test), 
                              validation_steps=50, 
                              verbose=1)